Theory

X - случ. величина количества попыток для перехода из одного состояние в другое. Она имеет геометрическое распределению, мат. ожидание которого есть E(X) = 1/p, где р - вероятность успеха на каждом шаге. 
Итого для данной цепи:
E(X) = E(X1) + E(X2) + E(X3) = 1/0.4 + 1/0.5 + 1/0.7 = 5.92 
то есть около 6 шагов

In [41]:
1/0.4 + 1/0.5 + 1/0.7

5.928571428571429

Practice

Task 1

In [1]:
import numpy as np
import os

filenames = os.listdir('np_data')
print(len(filenames))

5406


In [3]:
uids = list(set([fname.split('.')[0] for fname in filenames])) #utterance id ()
#print(uids[:5])

In [4]:
feats = {}
alis = {}
for uid in uids:
    feats[uid] = np.load('np_data/{}.feats.npy'.format(uid))
    alis[uid] = np.load('np_data/{}.targs.npy'.format(uid))

print(len(feats), len(alis))

2703 2703


get all silence tags

In [5]:
sils = set()
with open('./data/lang/phones/silence.txt', 'r', encoding='utf-8') as f:
    for line in f: 
        sils.add(line)
print(len(sils))

15


Get all phones tags

In [6]:
num2phone = {}
with open('./data/lang/phones.txt', 'r', encoding='utf-8') as f:
    for line in f:
        phone, num = line.split()
        num2phone[num] = phone
print(len(num2phone))

320


Simplified phones set

In [7]:
n2ph = {}
for num in num2phone:
    if num2phone[num] in sils:
        n2ph[num] = 'SIL'
    elif '_' in num2phone[num]:
        #main = num2phone[num].split('_')[0]
        #n2ph[num] = main
        n2ph[num] = num2phone[num].split('_')[0]
    else:
        n2ph[num] = num2phone[num]
        
print(len(n2ph))

320


Get triphones

In [8]:
from collections import OrderedDict
from collections import defaultdict
tri2f = defaultdict(list)

for uid in uids:
    ali = [n2ph[str(num)] for num in alis[uid]]
    ali.insert(0,'START')
    ali.append('END') #add start and end tags
    feat = list(feats[uid])
    feat.insert(0,'START')
    feat.append('END')
    unique = list(OrderedDict.fromkeys(ali)) #get ordered phone list
    pre, center, post = tuple(unique[:3])
    k = 2

    for i, a in enumerate(ali):
        if a == pre: 
            continue
        elif a == center:
            tri = tuple((pre, center, post))
            tri2f[tri] += [feat[i]] 
        elif a == post:
            if a == 'END':
                continue
            pre = center
            center = a     
            k += 1
            post = unique[k]
            tri = tuple((pre, center, post))
            tri2f[tri] += [feat[i]]

In [42]:
print(len(tri2f))

31117


Task 2

Get mu and sigma for triphones

In [9]:
def mu_sigma(feats):
    mean = np.mean(feats, axis=0)
    std = np.std(feats, axis=0)
    return mean, std

means = {}
stds = {}
for tri in tri2f:
    means[tri], stds[tri] = mu_sigma(tri2f[tri])

In [44]:
print(len(means[('SH', 'AE1', 'OW0')]))
print(len(stds[('SH', 'AE1', 'OW0')]))

40
40


In [11]:
tri2stat = defaultdict()
for tri in means:
    tri2stat[tri] = np.concatenate((means[tri],stds[tri]))

In [45]:
print(len(tri2stat[('SH', 'AE1', 'OW0')]))

80


Task 3

Triphone AE1

In [12]:
AE1 = {tri: tri2stat[tri] for tri in tri2stat if tri[1] == 'AE1'}
AE1_names = list(AE1.keys())
AE1_data = [AE1[tri] for tri in AE1_names]

Cluster

In [13]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=0).fit(AE1_data)
classes = kmeans.labels_
#results = zip(AE1_names, classes)

C:\Users\TK_adm\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [17]:
result = zip(list(classes), AE1_names)

In [26]:
clusters = defaultdict(list)
for i, cl in enumerate(list(classes)):
    clusters[cl].append(AE1_names[i]) 

In [38]:
print(len(AE1_names))
print(len(clusters[0]))
print(len(clusters[1]))
print(len(clusters[2]))
print(len(clusters[3]))
print(len(clusters[4]))

823
165
181
195
105
177


In [32]:
print(clusters[0])

[('SH', 'AE1', 'OW0'), ('UH1', 'AE1', 'IY0'), ('UH1', 'AE1', 'W'), ('AY1', 'AE1', 'AW1'), ('UW1', 'AE1', 'OW1'), ('M', 'AE1', 'V'), ('OY1', 'AE1', 'L'), ('T', 'AE1', 'IH1'), ('AY1', 'AE1', 'Z'), ('P', 'AE1', 'EU1'), ('P', 'AE1', 'Y'), ('OY1', 'AE1', 'AY1'), ('HH', 'AE1', 'JH'), ('V', 'AE1', 'JH'), ('NG', 'AE1', 'T'), ('NG', 'AE1', 'END'), ('AH0', 'AE1', 'HH'), ('S', 'AE1', 'NG'), ('V', 'AE1', 'D'), ('V', 'AE1', 'T'), ('M', 'AE1', 'NG'), ('G', 'AE1', 'IH1'), ('HH', 'AE1', 'W'), ('AH0', 'AE1', 'G'), ('N', 'AE1', 'EH1'), ('V', 'AE1', 'DH'), ('Y', 'AE1', 'NG'), ('DH', 'AE1', 'CH'), ('AY0', 'AE1', 'IH0'), ('TH', 'AE1', 'JH'), ('AA1', 'AE1', 'B'), ('B', 'AE1', 'M'), ('DH', 'AE1', 'EU0'), ('AO1', 'AE1', 'IH1'), ('V', 'AE1', 'IY1'), ('CH', 'AE1', 'D'), ('SH', 'AE1', 'IY0'), ('D', 'AE1', 'END'), ('R', 'AE1', 'G'), ('B', 'AE1', 'SH'), ('D', 'AE1', 'AY1'), ('DH', 'AE1', 'NG'), ('IY0', 'AE1', 'UH1'), ('EH1', 'AE1', 'Y'), ('IH0', 'AE1', 'EH1'), ('G', 'AE1', 'W'), ('EU1', 'AE1', 'D'), ('N', 'AE1', '

In [33]:
print(clusters[1])

[('DH', 'AE1', 'F'), ('B', 'AE1', 'JH'), ('F', 'AE1', 'TH'), ('IH0', 'AE1', 'AY1'), ('EH1', 'AE1', 'B'), ('B', 'AE1', 'F'), ('EY1', 'AE1', 'Y'), ('TH', 'AE1', 'CH'), ('UW1', 'AE1', 'AY1'), ('N', 'AE1', 'B'), ('CH', 'AE1', 'S'), ('F', 'AE1', 'SH'), ('IY0', 'AE1', 'AO1'), ('L', 'AE1', 'NG'), ('Z', 'AE1', 'S'), ('K', 'AE1', 'CH'), ('K', 'AE1', 'AH1'), ('T', 'AE1', 'TH'), ('HH', 'AE1', 'IH0'), ('L', 'AE1', 'HH'), ('L', 'AE1', 'Y'), ('SH', 'AE1', 'IH0'), ('AH0', 'AE1', 'AO1'), ('AY0', 'AE1', 'UW0'), ('V', 'AE1', 'CH'), ('M', 'AE1', 'F'), ('AH0', 'AE1', 'SH'), ('NG', 'AE1', 'G'), ('W', 'AE1', 'UW1'), ('AH1', 'AE1', 'NG'), ('T', 'AE1', 'V'), ('W', 'AE1', 'END'), ('V', 'AE1', 'F'), ('V', 'AE1', 'Y'), ('D', 'AE1', 'IH0'), ('HH', 'AE1', 'CH'), ('F', 'AE1', 'AH0'), ('OW1', 'AE1', 'L'), ('G', 'AE1', 'EH1'), ('OW1', 'AE1', 'K'), ('G', 'AE1', 'Y'), ('IY1', 'AE1', 'Z'), ('DH', 'AE1', 'IH0'), ('OW1', 'AE1', 'M'), ('EU1', 'AE1', 'JH'), ('M', 'AE1', 'T'), ('IH0', 'AE1', 'Y'), ('IH1', 'AE1', 'SH'), ('D',

In [34]:
print(clusters[2])

[('AW1', 'AE1', 'G'), ('HH', 'AE1', 'K'), ('L', 'AE1', 'G'), ('IY0', 'AE1', 'OW1'), ('P', 'AE1', 'Z'), ('B', 'AE1', 'D'), ('P', 'AE1', 'N'), ('Z', 'AE1', 'N'), ('R', 'AE1', 'AH0'), ('V', 'AE1', 'N'), ('AW1', 'AE1', 'S'), ('AY1', 'AE1', 'EY1'), ('IY0', 'AE1', 'S'), ('OW1', 'AE1', 'IY1'), ('AO1', 'AE1', 'D'), ('V', 'AE1', 'IH1'), ('IH0', 'AE1', 'N'), ('F', 'AE1', 'N'), ('B', 'AE1', 'UW1'), ('M', 'AE1', 'IH1'), ('N', 'AE1', 'F'), ('W', 'AE1', 'N'), ('SIL', 'AE1', 'L'), ('CH', 'AE1', 'K'), ('AY1', 'AE1', 'V'), ('Z', 'AE1', 'W'), ('R', 'AE1', 'D'), ('IH0', 'AE1', 'OW1'), ('AY1', 'AE1', 'N'), ('IH1', 'AE1', 'S'), ('N', 'AE1', 'G'), ('SH', 'AE1', 'HH'), ('UH1', 'AE1', 'END'), ('D', 'AE1', 'M'), ('L', 'AE1', 'AH1'), ('AY1', 'AE1', 'S'), ('IH0', 'AE1', 'S'), ('B', 'AE1', 'END'), ('EU0', 'AE1', 'K'), ('CH', 'AE1', 'AW1'), ('L', 'AE1', 'N'), ('Z', 'AE1', 'AH0'), ('T', 'AE1', 'N'), ('N', 'AE1', 'D'), ('DH', 'AE1', 'AY1'), ('Z', 'AE1', 'IH1'), ('EU0', 'AE1', 'CH'), ('AA1', 'AE1', 'HH'), ('IY1', 'AE

In [35]:
print(clusters[3])

[('T', 'AE1', 'AA1'), ('R', 'AE1', 'EY1'), ('EY1', 'AE1', 'EH1'), ('IY1', 'AE1', 'AO1'), ('IH1', 'AE1', 'OW1'), ('AY1', 'AE1', 'R'), ('IH1', 'AE1', 'N'), ('IH1', 'AE1', 'HH'), ('R', 'AE1', 'UH1'), ('D', 'AE1', 'AW1'), ('UH1', 'AE1', 'D'), ('AO1', 'AE1', 'IY0'), ('AY1', 'AE1', 'IY0'), ('N', 'AE1', 'Y'), ('IY0', 'AE1', 'HH'), ('EY1', 'AE1', 'AO1'), ('B', 'AE1', 'OW1'), ('OW1', 'AE1', 'G'), ('P', 'AE1', 'HH'), ('EU1', 'AE1', 'N'), ('B', 'AE1', 'G'), ('EY1', 'AE1', 'AH1'), ('AO1', 'AE1', 'L'), ('R', 'AE1', 'IH1'), ('W', 'AE1', 'F'), ('Z', 'AE1', 'UH1'), ('UW1', 'AE1', 'M'), ('JH', 'AE1', 'S'), ('V', 'AE1', 'AO1'), ('AW1', 'AE1', 'AH1'), ('IH1', 'AE1', 'AA1'), ('NG', 'AE1', 'M'), ('S', 'AE1', 'EH1'), ('F', 'AE1', 'UH1'), ('AW1', 'AE1', 'IH0'), ('OY1', 'AE1', 'D'), ('EY1', 'AE1', 'UW1'), ('TH', 'AE1', 'EU1'), ('M', 'AE1', 'Z'), ('V', 'AE1', 'HH'), ('IH1', 'AE1', 'UW1'), ('HH', 'AE1', 'SH'), ('SH', 'AE1', 'AO0'), ('Z', 'AE1', 'AO1'), ('OW1', 'AE1', 'N'), ('L', 'AE1', 'OW1'), ('AY1', 'AE1', 'F

In [31]:
print(clusters[4])

[('Y', 'AE1', 'AH1'), ('EU0', 'AE1', 'S'), ('HH', 'AE1', 'T'), ('SH', 'AE1', 'F'), ('K', 'AE1', 'END'), ('R', 'AE1', 'DH'), ('IH0', 'AE1', 'CH'), ('AH1', 'AE1', 'V'), ('EY1', 'AE1', 'G'), ('DH', 'AE1', 'T'), ('IY0', 'AE1', 'B'), ('F', 'AE1', 'EY1'), ('UW1', 'AE1', 'T'), ('UW0', 'AE1', 'END'), ('HH', 'AE1', 'AY1'), ('D', 'AE1', 'UW1'), ('K', 'AE1', 'B'), ('T', 'AE1', 'AH1'), ('Z', 'AE1', 'DH'), ('G', 'AE1', 'N'), ('OW1', 'AE1', 'V'), ('JH', 'AE1', 'END'), ('IY1', 'AE1', 'L'), ('R', 'AE1', 'P'), ('HH', 'AE1', 'Z'), ('JH', 'AE1', 'AA1'), ('DH', 'AE1', 'L'), ('HH', 'AE1', 'F'), ('K', 'AE1', 'AA1'), ('Z', 'AE1', 'K'), ('AE0', 'AE1', 'SH'), ('R', 'AE1', 'Y'), ('IY1', 'AE1', 'SH'), ('CH', 'AE1', 'F'), ('UW1', 'AE1', 'IY1'), ('Y', 'AE1', 'S'), ('AH0', 'AE1', 'V'), ('AH0', 'AE1', 'L'), ('R', 'AE1', 'SH'), ('Z', 'AE1', 'IH0'), ('P', 'AE1', 'S'), ('D', 'AE1', 'B'), ('M', 'AE1', 'P'), ('OW1', 'AE1', 'AO1'), ('HH', 'AE1', 'G'), ('T', 'AE1', 'B'), ('T', 'AE1', 'P'), ('P', 'AE1', 'IH0'), ('P', 'AE1',

Я теряюсь, как это все проинтерпретировать, так как ничего не знаю об английской фонетике.